<a href="https://colab.research.google.com/github/LucasBorba1/project_hailmary/blob/main/project_hailmary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>